In [1]:
# data processing packages
import numpy as np   
import pandas as pd 
from pandas import *
from numpy import *

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *
 
%matplotlib inline    
import matplotlib as mplt
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import IPython
from IPython.display import display

import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
# matplotlib.style.use('ggplot')
import itertools

import random

# machine leanring packages

from keras.models import Sequential 
from keras.layers.core import Dense, Activation, TimeDistributedDense, Dropout
from keras.layers.recurrent import LSTM
from keras.optimizers import *
from keras.callbacks import *
# RMSprop, Adadelta
from keras.regularizers import l2, activity_l2
from keras.utils.np_utils import to_categorical

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score

import xgboost as xgb


Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [2]:
# ------------data prepro-----------------------

files_list=["../dataset/dataset_demo/xtrain.csv", \
            "../dataset/dataset_demo/xtest.csv",\
            "../dataset/dataset_demo/ytrain.csv", \
            "../dataset/dataset_demo/ytest.csv"]

xtrain_df=pd.read_csv( files_list[0] ,sep=',', header=None)
xtest_df=pd.read_csv( files_list[1] ,sep=',', header=None)
ytrain_df=pd.read_csv( files_list[2] ,sep=',', header=None)
ytest_df=pd.read_csv( files_list[3] ,sep=',', header=None)

print xtrain_df.shape, xtest_df.shape, ytrain_df.shape, ytest_df.shape


# get training and testing data prepared

# cxtest = conti_normalization_test_dta(cxtest, cxtrain)
# cxtrain = conti_normalization_train_dta(cxtrain)

print xtrain_df.shape, ytrain_df.shape
print xtest_df.shape,  ytest_df.shape

# cxtrain = cxtrain.as_matrix() 
xtrain = xtrain_df.as_matrix()
xtest  = xtest_df.as_matrix()

ytrain = ytrain_df.as_matrix()
ytest  = ytest_df.as_matrix()

# ytest = np_utils.to_categorical( ytest_df.as_matrix() )
# ytrain =np_utils.to_categorical( ytrain_df.as_matrix() )

(6938, 186) (771, 186) (6938, 1) (771, 1)
(6938, 186) (6938, 1)
(771, 186) (771, 1)


In [7]:
def crossv_rf_num_estimatior(maxnum, X,Y):
    score_list=[]
    tmpscore=0
    tmpnum=0
    
    tmpy = Y.reshape( (len(Y),) )
    
    for i in range(1,maxnum+1,10):
        clf = RandomForestClassifier(n_estimators=i,max_depth=3)
        scores = cross_val_score(clf, X, tmpy)
        score_list.append(scores)
    
        if scores.mean()>tmpscore:
            tmpscore=scores.mean()
            tmpnum=i
    return tmpnum, score_list

def crossv_gbt_num_estimatior(maxnum,X,Y):
    score_list=[]
    tmpscore=0
    tmpnum=0
    
    tmpy = Y.reshape( (len(Y),) )
    
    for i in range(1,maxnum+1,10):
        clf = GradientBoostingClassifier(n_estimators=i,learning_rate=0.1)
        scores = cross_val_score(clf, X, tmpy)
        score_list.append(scores)
    
        if scores.mean()>tmpscore:
            tmpscore=scores.mean()
            tmpnum=i
    return tmpnum, score_list

def rf_num_estimatior(maxnum, X,Y, xtest, ytest):
    
    tmpy = Y.reshape( (len(Y),) )
    score = []
    for i in range(10,maxnum+1,10):
        clf = RandomForestClassifier(n_estimators=i,max_depth=3)
        
        clf.fit( X, tmpy )
        score.append( clf.score(xtest, ytest))
            
    return score

def gbt_num_estimatior(maxnum,X,Y, xtest, ytest):
    
    tmpy = Y.reshape( (len(Y),) )
    score = []
    for i in range(10,maxnum+1,10):
        clf = GradientBoostingClassifier(n_estimators=i,learning_rate=0.1)
        
        clf.fit( X, tmpy )
        score.append( clf.score(xtest, ytest))
    
    return score

def xgt_num_estimator( maxdepth, rounds ,X,Y, xtest, ytest):
    
    score = []
    xg_train = xgb.DMatrix(xtrain, label = ytrain)
    xg_test  = xgb.DMatrix(xtest,  label = ytest)

# setup parameters for xgboost
    param = {}
# use softmax multi-class classification
    param['objective'] = 'multi:softmax'
# scale weight of positive examples
    param['eta'] = 0.1
    param['max_depth'] = 0
    param['silent'] = 1
    param['nthread'] = 8
    param['num_class'] = 8
    
    for i in range(1, maxdepth):
        for num_round in range(1, rounds):
            
            param['max_depth'] = i
            bst = xgb.train(param, xg_train, num_round )
            
            pred = bst.predict( xg_test )
            tmp_accur = ( sum( int(pred[i]) == ytest[i] \
                             for i in range(len(ytest))) / float(len(ytest)) )
            
            score.append( (i,num_round,tmp_accur) )


In [45]:
def result_plot( accu_list, feature_name, maxnum, fig_title, filename ):
    fig = plt.figure()
    
    figsize=( 15.4,7)
    font_size=15
    fig.set_size_inches( figsize )
    matplotlib.rcParams.update({'font.size': font_size})

    axes = plt.gca()
    axes.set_ylim([0.2,0.35])
    
    x= range(1,maxnum+1,10)
    
    for i in range(len(accu_list)):
        
        tmp=[]
        for j in range(len(accu_list[i])):
            tmp.append( accu_list[i][j].mean() )
        
        plt.plot( x, tmp ,label=feature_name[i] )

    plt.title(fig_title)
    plt.ylabel('Accuray on validation datasets')
    plt.xlabel('# of tress or iterations')
    # plt.legend( loc='upper left',fontsize=12 )
    plt.legend(loc='upper right')

#     fig.savefig('./results/'+filename+'.jpg', format='jpg', bbox_inches='tight')


In [33]:
rf_accu= rf_num_estimatior(150,xtrain,ytrain,xtest,ytest)
print "Random forest best validation accuracy:", max(rf_accu)


Random forest best validation accuracy: 0.306095979248


In [14]:
gbt_accu = gbt_num_estimatior(150, xtrain,ytrain, xtest, ytest)
print "Gradient boosted tree best validation accuracy:", max(gbt_accu)


0.315175097276


In [ ]:
# gradient boosted tree

0.315175097276

In [8]:
xgt_accu = xgt_num_estimator(20, 100, xtrain,ytrain, xtest, ytest)
print "XGT best validation accuracy:", max(xgt_accu)

KeyboardInterrupt: 

In [52]:
# backup code
import xgboost as xgb
# read in data
xg_train = xgb.DMatrix(xtrain, label = ytrain)
xg_test  = xgb.DMatrix(xtest,  label = ytest)

# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 5
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 8

watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
num_round = 35
bst = xgb.train(param, xg_train, num_round, watchlist );
# get prediction
pred = bst.predict( xg_test );


print ('predicting, classification accuracy=%f' % \
       (sum( int(pred[i]) == ytest[i] for i in range(len(ytest))) / float(len(ytest)) ))


[0]	train-merror:0.613722	test-merror:0.721141
[1]	train-merror:0.579274	test-merror:0.709468
[2]	train-merror:0.561545	test-merror:0.709468
[3]	train-merror:0.545835	test-merror:0.692607
[4]	train-merror:0.536033	test-merror:0.687419
[5]	train-merror:0.531421	test-merror:0.684825
[6]	train-merror:0.518737	test-merror:0.677043
[7]	train-merror:0.515711	test-merror:0.67834
[8]	train-merror:0.509225	test-merror:0.688716
[9]	train-merror:0.501009	test-merror:0.693904
[10]	train-merror:0.496541	test-merror:0.684825
[11]	train-merror:0.492793	test-merror:0.67834
[12]	train-merror:0.487172	test-merror:0.688716
[13]	train-merror:0.479533	test-merror:0.688716
[14]	train-merror:0.472326	test-merror:0.683528
[15]	train-merror:0.466129	test-merror:0.690013
[16]	train-merror:0.460651	test-merror:0.692607
[17]	train-merror:0.454742	test-merror:0.692607
[18]	train-merror:0.452292	test-merror:0.69131
[19]	train-merror:0.445662	test-merror:0.696498
[20]	train-merror:0.442491	test-merror:0.70428
[21]	t